# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [1]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [2]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv")
customers = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv")

세 데이터프레임의 상위 5개 행을 조회하시오.

In [3]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [4]:
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [5]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [6]:
tmp = sales.groupby("ProductID", as_index = False)[["Qty","Amt"]].sum()
tmp

,ProductID,Qty,Amt
0,p1001771,1055,3354827
1,p1002841,903,11011541
2,p1005621,906,2601703
3,p1005771,3963,7319963
4,p1005891,5194,10119037
...,...,...,...
57,p1246581,2180,2456346
58,p1255281,809,955633
59,p1256521,701,1241545
60,p1284851,2350,6437323


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [7]:
tmp.sort_values("Amt", ascending=True).head()

,ProductID,Qty,Amt
54,p1234751,416,374057
7,p1011291,1775,479197
5,p1006281,625,536294
36,p1149391,721,623612
39,p1152861,617,672267


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [8]:
customers.groupby("Gender")["CustomerID"].count()

Gender
F    2060
M     183
Name: CustomerID, dtype: int64

### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [9]:
customers["Age"] = 2022 - customers["BirthYear"] +1
customers

,CustomerID,RegisterDate,Gender,BirthYear,Age
0,c328222,2014-09-25,F,1960,63
1,c281448,2013-06-18,F,1974,49
2,c038336,2003-10-10,F,1968,55
3,c084237,2007-03-09,F,1982,41
4,c162600,2010-06-14,F,1978,45
...,...,...,...,...,...
2238,c302101,2013-11-23,F,1981,42
2239,c213699,2011-11-04,F,1967,56
2240,c314668,2014-04-25,F,1977,46
2241,c160301,2010-05-15,F,1953,70


### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [10]:
bin = [0,30,40,50,60,70,np.inf]
label = ["30대미만","30대","40대","50대","60대","70대이상"]
customers["AgeGroup"] = pd.cut(customers["Age"], bins=bin, right=False, labels=label)

### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [11]:
customers.groupby("AgeGroup", as_index = False)[["CustomerID"]].count()

,AgeGroup,CustomerID
0,30대미만,3
1,30대,133
2,40대,859
3,50대,827
4,60대,299
5,70대이상,122


### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [12]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.

sales_product = pd.merge(sales, products, on="ProductID", how="inner")
sales_product

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,순두부,반찬류,두부,1
1,137,4,2016-01-02,p1036481,2,2100,c280590,순두부,반찬류,두부,1
2,63,16,2016-01-03,p1036481,1,1050,c037915,순두부,반찬류,두부,1
3,135,3,2016-01-04,p1036481,3,3150,c100815,순두부,반찬류,두부,1
4,63,13,2016-01-06,p1036481,10,10500,c048405,순두부,반찬류,두부,1
...,...,...,...,...,...,...,...,...,...,...,...
70007,89,1,2017-03-31,p1299491,1,2150,c397789,당근,채소,뿌리채소,5
70008,69,5,2017-03-31,p1299491,1,2150,c268330,당근,채소,뿌리채소,5
70009,121,4,2017-03-31,p1299491,1,2150,c222461,당근,채소,뿌리채소,5
70010,159,9,2017-03-31,p1299491,1,2150,c275305,당근,채소,뿌리채소,5


In [13]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.

sales_product.groupby("ProductName", as_index= False)[["Amt"]].sum().head()

,ProductName,Amt
0,감귤컵푸딩,677154
1,감자칩,3676518
2,고구마스틱,1620986
3,깻잎,2930359
4,낱개캔디,479197


In [14]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.

sales_product.groupby("ProductName", as_index= False)[["Amt"]].sum().sort_values("Amt", ascending=True).head()

,ProductName,Amt
47,초코콘,374057
4,낱개캔디,479197
27,소보루빵,536294
16,배아이스크림,623612
56,포토아이스크림,672267


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [15]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.

sales_cust = pd.merge(sales, customers, on="CustomerID", how="inner")
sales_cust.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,49,40대
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,49,40대
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,49,40대
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,49,40대
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,49,40대


In [16]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.

sales_cust.groupby("AgeGroup", as_index = False)[["Amt"]].sum()

,AgeGroup,Amt
0,30대미만,127712
1,30대,7311268
2,40대,75865355
3,50대,81033746
4,60대,21990389
5,70대이상,8670998


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [17]:
# 1) 세 데이터프레임을 merge 

tmp = pd.merge(sales, customers, how="inner")
scp = pd.merge(tmp, products, how="inner")
scp.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,49,40대,순두부,반찬류,두부,1
1,197,5,2017-01-24,p1036481,1,1050,c150417,2010-03-03,F,1974,49,40대,순두부,반찬류,두부,1
2,81,5,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1
3,81,6,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1
4,23,13,2016-10-16,p1036481,1,1050,c350776,2015-06-06,F,1983,40,40대,순두부,반찬류,두부,1


In [18]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다

agegroup = ['30대미만', '30대', '40대', '50대', '60대', '70대이상']

for name in agegroup:
    print(f"#####{name}의 카테고리별 비중#####")
    scp_ct = scp.loc[scp["AgeGroup"] == name]
    scp_ct = scp_ct.groupby("Category", as_index = False)[["Amt"]].sum()
    scp_ct.rename(columns={'Amt' : name}, inplace=True)
    scp_ct["카테고리별 비중"] = scp_ct[name] / scp_ct[name].sum() * 100
    display(scp_ct)

#####30대미만의 카테고리별 비중#####


,Category,30대미만,카테고리별 비중
0,간식,10277,8.047012
1,과일,8100,6.342395
2,반찬류,29692,23.249186
3,유제품,18829,14.743329
4,채소,60814,47.618078


#####30대의 카테고리별 비중#####


,Category,30대,카테고리별 비중
0,간식,508425,6.953992
1,과일,1786840,24.439536
2,반찬류,961119,13.145722
3,유제품,2142519,29.304342
4,채소,1912365,26.156407


#####40대의 카테고리별 비중#####


...

,Category,50대,카테고리별 비중
0,간식,5605234,6.917160
1,과일,20550417,25.360320
2,반찬류,13336189,16.457574
3,유제품,18812249,23.215327
4,채소,22729657,28.049619


#####60대의 카테고리별 비중#####


,Category,60대,카테고리별 비중
0,간식,822619,3.740811
1,과일,4995552,22.716979
2,반찬류,5118525,23.276191
3,유제품,4084638,18.574651
4,채소,6969055,31.691368


#####70대이상의 카테고리별 비중#####


,Category,70대이상,카테고리별 비중
0,간식,219339,2.529570
1,과일,2463150,28.406765
2,반찬류,2350672,27.109590
3,유제품,910932,10.505504
4,채소,2726905,31.448571
